## Day 47 Lecture 2 Assignment

In this assignment, we will perform K-Medoids clustering using responses to a survey about student life at a university.

In [3]:
!pip install pyclustering

     |████████████████████████████████| 2.6MB 2.8MB/s 
  Created wheel for pyclustering: filename=pyclustering-0.9.2-cp36-none-any.whl size=2585545 sha256=e86db78050cd052f026144125a92c9e568a30f091a43906d87a955d5f9260c6e
  Stored in directory: /root/.cache/pip/wheels/64/8c/62/f45c1df20df691f3dfb2c71be53660cf5d46a98f5c3fa53cec
Successfully built pyclustering


In [0]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from pyclustering.cluster.kmedoids import kmedoids
import random

This dataset consists of 35 binary features, each corresponding to a yes/no question that characterizes the student taking the survey.

This dataset contains a large number of features, each corresponding to a survey question. The feature name summarizes the survey question, so we will not list them all out here.

Load the dataset.

In [5]:
# answer goes here
df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/student_life_survey.csv')
df.head()



,Q2-Participated in Societies and Interest Groups,Q2-Participated in Clubs,"Q2-Participated in Halls, JCRCs and/or Residential College CSCs",Q2-Participated in University organised events,Q3-Interested in Arts & Culture,Q3-Interested in Science & Technology,Q3-Interested in Research and independent study,Q3-Interested in Sports,"Q3-Interested in Other competitions (eg case, debates)",Q3-Interested in Entrepreneurship,Q3-Interested in Volunteering,Q3-Interested in Others,Q4-Passionate about Animal welfare,Q4-Passionate about Arts/Culture/Heritage,Q4-Passionate about Children/Youth,Q4-Passionate about Community building,"Q4-Passionate about Diversity & Inclusion (e.g. special needs, migrant worker, interfaith and intercultural understanding)",Q4-Passionate about Environmental sustainability,Q4-Passionate about Families,Q4-Passionate about Health/Well-being (e.g mental health),Q4-Passionate about Seniors,Q4-Passionate about Poverty reduction,Q4-Passionate about Education,Q4-Passionate about None of the above,Q4-Passionate about Others,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,response_id
0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,2
2,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,3
3,1,1,1,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,1,1,1,1,0,1,0,4
4,1,0,1,1,0,1,1,0,0,1,1,0,0,0,1,1,0,1,0,1,0,0,1,0,0,0,1,1,0,1,0,0,0,1,0,5


For our analysis, we will focus on a specific subset of the survey that is focused on stress. These questions all begin with the string 'Q5'. Filter the columns that meet this criteria (should be 10 in total).

In [6]:
df_filtered = df.filter(regex ='Q5'  )
df_filtered.head()
df_filtered.shape

(2958, 10)

The pyclustering implementation of kmedoids supports a variety of distance metrics, but they are primarily for numeric data. We will be using SMC/Hamming dissimilarity and precomputing the similarity matrix (an alternative would be to specify a user-defined function, which you are welcome to try in addition).

We'll assume for the next step that a pair of negative values (i.e. both responses are "no") is as informative as a pair of positive values. Compute the full distance/dissimilarity matrix for the survey data using matching/hamming distance.

In [7]:
# answer goes here
matrix_dist=squareform(pdist(df_filtered,'hamming'))
print(matrix_dist)
matrix_df = pd.DataFrame(matrix_dist)
# matrix_df.head()

[[0.  0.  0.1 ... 0.  0.2 0.2]
 [0.  0.  0.1 ... 0.  0.2 0.2]
 [0.1 0.1 0.  ... 0.1 0.1 0.1]
 ...
 [0.  0.  0.1 ... 0.  0.2 0.2]
 [0.2 0.2 0.1 ... 0.2 0.  0.2]
 [0.2 0.2 0.1 ... 0.2 0.2 0. ]]


Using the dissimilarity matrix, perform kmedoids clustering using k=2. Set the initial medoids randomly. Note that pyclustering expects the distance matrix to be a numpy array; a pandas dataframe may cause errors. 

Which survey responses are chosen as the cluster representatives? Print out the details of these responses.

In [28]:
n_clusters = 2
initial_medoids = random.sample(list(matrix_df.index), n_clusters)
initial_medoids

[959, 61]

In [13]:
# Create instance of K-Medoids algorithm.
kmedoids_instance = kmedoids(matrix_df , initial_medoids)
# Run cluster analysis and obtain results.
kmedoids_instance.process()
clusters = kmedoids_instance.get_clusters()
kmedoids_instance.get_medoids()


[2, 656]

In [29]:
df_filtered.iloc[656]

Q5-Stressed about Adjustment issues                                                    1
Q5-Stressed about Academic issues                                                      1
Q5-Stressed about Financial issues                                                     1
Q5-Stressed about Family issues                                                        0
Q5-Stressed about Friendships                                                          1
Q5-Stressed about Romantic relationships                                               0
Q5-Stressed about  Health related issues                                               0
Q5-Stressed about Career related issues                                                0
Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.    1
Q5-Stressed about Others                                                               0
Name: 656, dtype: int64

If you run the previous cell a few times, you'll probably notice that the medoids are very sensitive to initialization. A common approach to produce well-separated clusters is to choose initial centroids that are far apart. Re-run the previous process, except with a random pair of centroids that have a dissimilarity of 0.8 or higher. Are the results more stable now? How would you describe the typical clusters you see?

In [25]:
arrays = list(np.where(matrix_dist>.8))
arrays

[array([   3,    3,    3, ..., 2933, 2939, 2953]),
 array([ 183,  456,  708, ...,  628, 2899, 2416])]

In [0]:
matrix_dict = dict(zip(arrays[0], arrays[1]))

In [30]:
random_centroid = random.choice(list(matrix_dict.items()))
random_centroid

(930, 628)

In [31]:
# Create instance of K-Medoids algorithm.
kmedoids_instance = kmedoids(matrix_df , random_centroid)
# Run cluster analysis and obtain results.
kmedoids_instance.process()
clusters = kmedoids_instance.get_clusters()
kmedoids_instance.get_medoids()


[297, 634]

The results are more stable when the dissimilarity is greater and most respondents where stressed about adjustment, academic, and financial issues.

In [32]:
df_filtered.iloc[634]

Q5-Stressed about Adjustment issues                                                    1
Q5-Stressed about Academic issues                                                      1
Q5-Stressed about Financial issues                                                     1
Q5-Stressed about Family issues                                                        0
Q5-Stressed about Friendships                                                          1
Q5-Stressed about Romantic relationships                                               1
Q5-Stressed about  Health related issues                                               0
Q5-Stressed about Career related issues                                                1
Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.    1
Q5-Stressed about Others                                                               0
Name: 634, dtype: int64